In [0]:
from google.colab import drive
drive.mount('/content/drive/')

In [0]:
import pandas as pd
import numpy as np
dataset = pd.read_csv("https://raw.githubusercontent.com/mwitiderrick/kerasDO/master/HR_comma_sep.csv")


In [5]:
dataset.tail(5)

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,department,salary
14994,0.40,0.57,2,151,3,0,1,0,support,low
14995,0.37,0.48,2,160,3,0,1,0,support,low
14996,0.37,0.53,2,143,3,0,1,0,support,low
14997,0.11,0.96,6,280,4,0,1,0,support,low
14998,0.37,0.52,2,158,3,0,1,0,support,low


In [6]:
# One hot encoding of categorical values such as depertment and salary as we can see
# They are not numbers and the model can't process them as strings
feats = ['department','salary']
final = pd.get_dummies(dataset,columns=feats,drop_first=True)
final.head()



,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,department_RandD,department_accounting,department_hr,department_management,department_marketing,department_product_mng,department_sales,department_support,department_technical,salary_low,salary_medium
0,0.38,0.53,2,157,3,0,1,0,0,0,0,0,0,0,1,0,0,1,0
1,0.80,0.86,5,262,6,0,1,0,0,0,0,0,0,0,1,0,0,0,1
2,0.11,0.88,7,272,4,0,1,0,0,0,0,0,0,0,1,0,0,0,1
3,0.72,0.87,5,223,5,0,1,0,0,0,0,0,0,0,1,0,0,1,0
4,0.37,0.52,2,159,3,0,1,0,0,0,0,0,0,0,1,0,0,1,0


In [0]:
# Now we have to prepare our model for the training so we use the left as category to know if they left

from sklearn.model_selection import train_test_split
X = final.drop(['left'],axis=1).values
y = final['left'].values
# Now we split the data depending on our preference
# In this case we will use 80 - 20 spliting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [62]:
for num,x in enumerate(X):
    print(num, x) if num == 1 else x

1 [  0.8    0.86   5.   262.     6.     0.     0.     0.     0.     0.
   0.     0.     0.     1.     0.     0.     0.     1.  ]


In [0]:
# We can now use a scaler privided by scikit-learn to scale our data so that none
# of it matters more than the other for our model

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


In [0]:
# We will proceed with the construction of our model
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout

### Adding a Dropout Layer helps us to generalize more by helping to reduce the overfitting

In [14]:

classifier = Sequential()
classifier.add(Dense(9, kernel_initializer = "uniform", activation = "relu", input_dim=18))
classifier.add(Dropout(rate = 0.1))
classifier.add(Dense(1, kernel_initializer = "uniform", activation = "sigmoid"))
classifier.compile(optimizer= "adam",loss = "binary_crossentropy",metrics = ["accuracy"])
classifier.fit(X_train, y_train, batch_size = 10, epochs = 1)
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Epoch 1/1
11249/11249 [==============================] - 3s 285us/step - loss: 0.4277 - acc: 0.7986


array([[2756,   92],
       [ 548,  354]])

## (2756+354)/3750=82.9

Now we will use K-Folds to get better accuracy by doing cross-validation with different parts of our dataset

In [0]:
from sklearn.model_selection import GridSearchCV
def make_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(9, kernel_initializer = "uniform", activation = "relu", input_dim=18))
    classifier.add(Dense(1, kernel_initializer = "uniform", activation = "sigmoid"))
    classifier.compile(optimizer= optimizer,loss = "binary_crossentropy",metrics = ["accuracy"])
    return classifier

In [0]:
classifier = KerasClassifier(build_fn = make_classifier)

In [0]:
params = {
    'batch_size':[20,35],
    'epochs':[2,3],
    'optimizer':['adam','rmsprop']
}

In [0]:
grid_search = GridSearchCV(estimator=classifier,
                           param_grid=params,
                           scoring="accuracy",
                           cv=2)

In [37]:
grid_search = grid_search.fit(X_train,y_train)

Epoch 1/2
5624/5624 [==============================] - 1s 191us/step - loss: 0.5668 - acc: 0.7630
Epoch 2/2
5624/5624 [==============================] - 1s 146us/step - loss: 0.3724 - acc: 0.8405
Epoch 1/2
5625/5625 [==============================] - 1s 190us/step - loss: 0.5728 - acc: 0.7676
Epoch 2/2
5625/5625 [==============================] - 1s 128us/step - loss: 0.3924 - acc: 0.7995
Epoch 1/2
5624/5624 [==============================] - 1s 157us/step - loss: 0.5696 - acc: 0.7628
Epoch 2/2
5624/5624 [==============================] - 1s 117us/step - loss: 0.4033 - acc: 0.7959
Epoch 1/2
5625/5625 [==============================] - 1s 161us/step - loss: 0.5718 - acc: 0.7620
Epoch 2/2
5625/5625 [==============================] - 1s 120us/step - loss: 0.4303 - acc: 0.7625
Epoch 1/3
5624/5624 [==============================] - 1s 186us/step - loss: 0.5740 - acc: 0.7697
Epoch 2/3
5624/5624 [==============================] - 1s 131us/step - loss: 0.3804 - acc: 0.8121
Epoch 3/3
5624/5624 

In [38]:
best_param = grid_search.best_params_
best_accuracy = grid_search.best_score_
best_param

{'batch_size': 20, 'epochs': 3, 'optimizer': 'adam'}

In [39]:
best_accuracy

0.8644323939905769

0.8644323939905769### As we can see, we have improved in small steps the performance of our model, with K-Folds we
### have the hability to do some more training to explore the data and validate with it if we don't have
### a huge dataset like ours

## Increasing accuracy to 0.8644323939905769%

In [0]:
new_pred_neg = grid_search.predict(sc.transform(np.array([[  0.56,   0.58,  4.,   258.,     3.,     0.,     0.,     0.,     0.,     0.,   0.,     0.,     0.,     1.,     0.,     0.,     0.,     1.,  ]])))
new_pred_pos = grid_search.predict(sc.transform(np.array([[  0.8,    0.86,   5.,   262.,     6.,     0.,     0.,     0.,     0.,     0.,   0.,     0.,     0.,     1.,     0.,     0.,     0.,     1.  ]])))


In [64]:
# new_pred_pos = new_pred_pos > 0.5
new_pred_pos

array([[1]])